In [1]:
import pandas as pd

In [19]:
def payer_category(x):
    payers = {
        'Charge': 'gross',
        'Self Pay Price': 'cash',
        'Min Negotiated Rate': 'min',
        'Max Negotiated Rate': 'max',
    }

    try:
        return payers[x]
    except KeyError:
        return 'payer'

In [2]:
file = '540519577_mary-washington-hospital_standardcharges.xlsx'
df = pd.read_excel(file)

In [4]:
df.rename(columns={
    'Code': 'code',
    'Desc.': 'description'
}, inplace=True)

In [14]:
cols = df.columns.tolist()
id_vars = ['code', 'description']
value_vars = cols[3:]
value_vars.append('Charge')

df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer', value_name= 'standard_charge')

C:\Users\adria\AppData\Local\Temp\ipykernel_22480\1417882709.py:6: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer', value_name= 'standard_charge')


KeyError: "The following 'value_vars' are not present in the DataFrame: ['Charge']"

In [15]:
df.dropna(subset=['standard_charge'], inplace=True)

In [17]:
df.loc[df['code'].str.len() == 5, 'hcpcs_cpt'] = df['code']

In [22]:
df['payer_category'] = df['payer'].apply(payer_category)

In [26]:
ccn = {
    '540519577': '490022',
    '134316364': '490140'
}

ein = file.split('_')[0]
df['hospital_id'] = ccn[ein]

In [27]:
df.to_csv(file.split('_')[0] + '.csv', index=False)